<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TERM3/ReinforcementLearning/RL-ASSIGNMENT2/CH24M571_AASIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Soumya Mukherjee - ch24m571

In [1]:
!pip install beautifulsoup4 requests


In [4]:
# imports
from collections import defaultdict, Counter
from typing import List, Tuple, Dict
import requests
from bs4 import BeautifulSoup
import numpy as np
import random
from math import log2

In [25]:


# -----------------
# Load words function
# -----------------
def load_words(file_path: str = "words.txt") -> List[str]:
    '''
    This will return back a list of strings read from a local file,
    filtered by length (4 letters) and with duplicates removed.
    '''
    words = []
    try:

      with open("words.txt", 'r') as f:
        for line in f:
            # Split the line by whitespace and add each word to the list
            for word in line.split():
                if len(word) == 4 and len(set(word)) == 4: # Filter for 4-letter unique-letter words
                    words.append(word)

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return []

    return list(set(words))


# -----------------
# Feedback function
# -----------------
def cats_dogs(secret: str, guess: str) -> Tuple[int,int]:
    """Return (cats, dogs) feedback between secret and guess.
       dogs = correct letter in correct position
       cats = correct letter but wrong position
    """
    dogs = sum(s == g for s,g in zip(secret, guess))
    cats = sum((Counter(secret) & Counter(guess)).values()) - dogs
    return cats, dogs

def is_consistent(candidate: str, guess: str, feedback: Tuple[int,int]) -> bool:
    """Check if candidate word is consistent with given guess+feedback"""
    return cats_dogs(candidate, guess) == feedback

# -----------------
# Minimax strategy (expensive but strong)
# -----------------
def partition_sizes_for_guess(guess: str, candidates: List[str]) -> Dict[Tuple[int,int], int]:
    buckets = defaultdict(int)
    for c in candidates:
        buckets[cats_dogs(c, guess)] += 1
    return buckets

def next_guess_minimax(candidates: List[str]) -> str:
    """Choose the guess that minimizes the worst-case remaining candidates."""
    best_guess, best_score = None, float('inf')
    for g in candidates:
        worst = max(partition_sizes_for_guess(g, candidates).values())
        if worst < best_score:
            best_score, best_guess = worst, g
    return best_guess

# -----------------
# Overlap heuristic (light + fast)
# -----------------
def likely_fixed_positions(candidates: List[str]):
    """Identify positions that are fixed (all candidates agree on the letter)."""
    fixed = {}
    for i in range(len(candidates[0])):
        letters = {w[i] for w in candidates}
        if len(letters) == 1:
            fixed[i] = next(iter(letters))
    return fixed


def next_guess_overlap(candidates: List[str]) -> str:
    """Pick guess biased by letter frequency and fixed 'dog' positions."""
    letter_counts = Counter("".join(candidates))
    fixed = likely_fixed_positions(candidates)

    def score(word):
        s = sum(letter_counts[ch] for ch in set(word))
        for i, ch in fixed.items():
            if word[i] == ch:
                s += 5  # reward matching known fixed positions
        return s

    # Ensure candidates is not empty before calling max
    if not candidates:
        return ""
    return max(candidates, key=score)


# -----------------
# Hybrid controller
# -----------------
def hybrid_next_guess(candidates: List[str], step: int, minimax_steps: int = 2, switch_threshold: int = 300) -> str:
    """Hybrid: use minimax in first steps or while candidates are large, then switch."""
    if not candidates:
      return "" # Return empty string if candidates is empty

    if step <= minimax_steps or len(candidates) > switch_threshold:
        return next_guess_minimax(candidates)
    else:
        return next_guess_overlap(candidates)

# -----------------
# Full play loop
# -----------------
def solve_secret(words: List[str], secret: str, minimax_steps: int = 2, switch_threshold: int = 300):
    """Simulate solving the secret with hybrid strategy."""
    candidates = words[:]
    step = 1
    history = []

    while True:
        guess = hybrid_next_guess(candidates, step, minimax_steps, switch_threshold)
        if not guess: # Check if guess is empty due to no candidates
            print("No possible words left. Could not solve.")
            break

        cats, dogs = cats_dogs(secret, guess)
        history.append((step, guess, cats, dogs, len(candidates)))

        if guess == secret:
            break

        # prune inconsistent candidates
        candidates = [c for c in candidates if is_consistent(c, guess, (cats, dogs))]
        step += 1

    return history

# -----------------
# Example usage
# -----------------
if __name__ == "__main__":
    # Example word list (replace with your full list)
    words = load_words()

    print(f"Loaded {len(words)} valid words")

    if len(words) > 0:
        secret = "SEXY" # You can change this to any word from your list
        if secret in words:
            result = solve_secret(words, secret)

            for step, guess, cats, dogs, remaining in result:
                print(f"Step {step}: Guess={guess}, Cats={cats}, Dogs={dogs}, Remaining={remaining}")

            print(f"Solved in {len(result)} steps! Secret was {secret}")
        else:
            print(f"Secret word '{secret}' not found in the loaded word list.")
    else:
        print("No valid words loaded. Cannot run the game.")

Loaded 4309 valid words
Step 1: Guess=TARS, Cats=1, Dogs=0, Remaining=4309
Step 2: Guess=POET, Cats=1, Dogs=0, Remaining=1017
Step 3: Guess=EINA, Cats=1, Dogs=0, Remaining=265
Step 4: Guess=SLUE, Cats=1, Dogs=1, Remaining=69
Step 5: Guess=URDE, Cats=1, Dogs=0, Remaining=7
Step 6: Guess=SEXY, Cats=0, Dogs=4, Remaining=2
Solved in 6 steps! Secret was SEXY


In [17]:
# from typing import List

# def load_words(file_path: str = "words.txt") -> List[str]:
#     '''
#     This will return back a list of strings read from a local file,
#     filtered by length (4 letters) and with duplicates removed.
#     '''
#     words = []
#     try:
#         with open(file_path, 'r') as f:
#             for line in f:
#                 word = line.strip().upper()
#                 if len(word) == 4 and len(set(word)) == 4: # Filter for 4-letter unique-letter words
#                     words.append(word)
#     except FileNotFoundError:
#         print(f"Error: The file '{file_path}' was not found.")
#         return []

#     return list(set(words))

# # Example usage (you can replace this with your main execution block)
# # if __name__ == "__main__":
# #     words = load_words()
# #     print(f"Loaded {len(words)} valid words from the file.")
# #     # You can now use the 'words' list in your game logic
# #     # For example, you could call your solve_secret function here
# #     # if len(words) > 0:
# #     #     secret = random.choice(words) # Choose a random secret word
# #     #     print(f"Secret word: {secret}")
# #     #     result = solve_secret(words, secret)
# #     #     for step, guess, cats, dogs, remaining in result:
# #     #         print(f"Step {step}: Guess={guess}, Cats={cats}, Dogs={dogs}, Remaining={remaining}")
# #     #     print(f"Solved in {len(result)} steps!")
# #     # else:
# #     #      print("No valid words loaded from the file. Cannot run the game.")

In [22]:
# all_words = []
# try:
#     with open("words.txt", 'r') as f:
#         for line in f:
#             # Split the line by whitespace and add each word to the list
#             for word in line.split():
#                 all_words.append(word.strip())
# except FileNotFoundError:
#     print("Error: the file 'words.txt' was not found.")

# print(f"Initially Loaded {len(all_words)} words from words.txt")
# all_words = list(set(all_words))
# print(f"filtered {len(all_words)} words from words.txt")

Initially Loaded 5526 words from words.txt
filtered 5526 words from words.txt
